In [1]:
import numpy as np
import mlflow.sklearn
from mlflow import MlflowClient
from sklearn.linear_model import LinearRegression


def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print("run_id: {}".format(r.info.run_id))
    print("artifacts: {}".format(artifacts))
    print("params: {}".format(r.data.params))
    print("metrics: {}".format(r.data.metrics))
    print("tags: {}".format(tags))


# prepare training data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Auto log all the parameters, metrics, and artifacts
mlflow.autolog()
model = LinearRegression()
with mlflow.start_run() as run:
    model.fit(X, y)

# fetch the auto logged parameters and metrics for ended run
print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))

2023/06/19 09:48:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/06/19 09:48:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/hericson/anaconda3/envs/my_projects/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


run_id: a730f3429c9f4e20881836db18adf0a9
artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/model.pkl', 'model/python_env.yaml', 'model/requirements.txt']
params: {'copy_X': 'True', 'fit_intercept': 'True', 'n_jobs': 'None', 'positive': 'False'}
metrics: {'training_mean_squared_error': 1.9721522630525295e-31, 'training_mean_absolute_error': 2.220446049250313e-16, 'training_r2_score': 1.0, 'training_root_mean_squared_error': 4.440892098500626e-16, 'training_score': 1.0}
tags: {'estimator_name': 'LinearRegression', 'estimator_class': 'sklearn.linear_model._base.LinearRegression'}
